# **GEOS2115/2915 2023 Practical Weeks 3 and 4:** Historical and future climate change in observations and CMIP6 models.

In weeks 3 and 4 of the practicals for GEOS2115 we will be analysing output from simulations run as part of the World Climate Research Program (WCRP)'s Coupled Model Intercomparison Program (CMIP6). We will compare these simulations to some observations, look at their representation of historical climate change and their projections/predictions of how the climate may change in the future.

This practical is split into two parts:
- Week 3: Introduction to analysing and plotting the CMIP6 data \& historical climate in ACCESS-CM2.
- Week 4: Projected future climate change in CMIP6

This notebook is for Week 4.

# Week 4: Projected Future climate change in CMIP6

In the week 3 practical we explored how climate changes over the historical period (1850-2014) were represented in the ACCESS-CM2 climate model, and how they compared to observations. This week, we will instead look forward into the (possible) future. We will do this using CMIP6 simulations where carbon dioxide concentrations (and other greenhouse gas and aerosol concentrations), follow several different possible futures based on "Shared Socio-economic pathways (SSPs)".

The SSPs were introduced in the Week 2 lecture. A detailed explainer of what the SSPs are and how they were generated can be found at https://www.carbonbrief.org/explainer-how-shared-socioeconomic-pathways-explore-future-climate-change/

In this practical we have provided data from simulations run with forcing from two SSPs:
- SSP2-4.5: This is a "middle of the road" scenario where we manage to stabilize our CO2 emissions at about current levels until 2050, after which they fall reaching close to zero by 2100 (gray line in figure below).
- SSP5-8.5: This is a "fossil-fueled development" scenario where emissions continue reaching triple their pre-industrial value by 2075 before stabilizing (red line in figure below).

<div>
<center><img src="images/ssps.png" width="600"/>
    </center>
    <p style="font-size:10px;">Image credit: https://www.carbonbrief.org/cmip6-the-next-generation-of-climate-models-explained/</p>
</div>


## GMST response in SSP2-4.5 and SSP5-8.5

Let's start out by looking at what the global mean surface temperature (GMST) response in the ACCESS-CM2 model is in the SSP2-4.5 scenario.

First, let's load the modules we need, and the historical and SSP245 ACCESS-CM2 datasets:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
xr.set_options(keep_attrs=True) # Keep attributes like units through calculations
import cmocean as cm

In [2]:
# Load historical and ssp245 datasets:
ACCESS_CM2_hist = xr.open_dataset('/course/data/ACCESS-CM2_hist_data.nc')
ACCESS_CM2_ssp245 = xr.open_dataset('/course/data/ACCESS-CM2_ssp245_data.nc')

We would like to plot the GMST change from these simulations. To do this we first need to define the `global_average` function, as we did in Week 3, in order to calculate the global mean surface temperature from the model output temperature defined at each longitude and latitude.

In [3]:
# Define a function to calculate the global average of the variable "variable" 
# using an equal-area weighting according to the "area" dataset
def global_average(variable,area):
    # Calculate the total area:
    total_area = area.where(variable.notnull()).sum('longitude').sum('latitude')
    
    # Calculate the global average by multiplying the variable by the 
    # grid cell area at each grid cell, summing, and then dividing by the total area:
    average = (variable*area).sum('longitude').sum('latitude')/total_area
    
    return(average)

We could now plot and calculate the GMST in both the historical and ssp245 simulations. However, as discussed in Week 3, for the purposes of comparing between models (which we will do later in this practical) and understanding the *changes* in GMST rather than its absolute magnitude, it is useful to first subtract the temperature averaged over an early period (here 1900-1950). This period is known as a baseline. Note: The choice of exactly what baseline to choose can be a big issue and has been the subject of entire research articles (e.g. https://iopscience.iop.org/article/10.1088/1748-9326/aba3d7).

Here, we define a function to calculate this baseline period, as we will use this calculation multiple times:

In [4]:
# Define baseline period:
bline_per = [1900,1950]

# Define a function to calculate the mean of a variable "var" over the baseline period:
def calc_baseline(var):
    baseline = var.sel(year=slice(bline_per[0],bline_per[1])).mean('year')
    return(baseline)

We can now calculate the GMST change in both the ACCESS-CM2 historical and SSP2-4.5 scenarios, and then plot them:

In [5]:
# Calculate the average temperature over the baseline period from the historical simulation:
ACCESS_CM2_GMST_baseline = calc_baseline(global_average(ACCESS_CM2_hist.SAT,ACCESS_CM2_hist.area))

# Calculate GMST change in both historical and ssp245 experiments by subtracting the baseline:
ACCESS_CM2_hist_GMST = global_average(ACCESS_CM2_hist.SAT,ACCESS_CM2_hist.area) - ACCESS_CM2_GMST_baseline
ACCESS_CM2_ssp245_GMST = global_average(ACCESS_CM2_ssp245.SAT,ACCESS_CM2_ssp245.area) - ACCESS_CM2_GMST_baseline

In [6]:
# Plot the curves on one plot:
ACCESS_CM2_hist_GMST.plot(label='ACCESS-CM2 historical')
ACCESS_CM2_ssp245_GMST.plot(label='ACCESS-CM2 SSP2-4.5')

plt.title('Global Mean Surface Temperature')
plt.legend()
plt.grid() # Add grid lines

According to ACCESS-CM2, the planet warms by about 1C up to 2014 within the historical period. Then, in the SSP2-4.5 experiment it warms a further 2.5C by 2100. Since anthropogenic carbon dioxide emissions in the SSP2-4.5 scenario drop back toward zero by 2100, the surface temperature appears to stabilize at about +3.5C toward the end of the century

<font color='red'> 
    
21. By adding additional code above, load, calculate and then plot the GMST for the ACCESS-CM2 SSP5-8.5 scenario (along with the historical and SSP2-4.5 scenarios on the same plot as above). Include this plot in your report. Describe the differences that you see between the two scenarios. Also comment on the magnitude and rate of GMST change in the future scenarios compared to during the historical period. **[3 marks]** 
</font>

## Regional changes in surface temperature and precipitation

Let's now take a look at the regional pattern of these changes.

### Surface temperature

The below code block calculates the spatial structure of the SAT in the years 2080-2100 in the SSP2-4.5 scenario, subtracts the baseline temperature over the period 1900-1950 from the historical simulation, and then plots it.

In [28]:
# Calculate SAT averaged over the baseline and modern periods:
ACCESS_CM2_SAT_baseline = calc_baseline(ACCESS_CM2_hist.SAT)
ACCESS_CM2_SAT_ssp245_21C = ACCESS_CM2_ssp245.SAT.sel(year=slice(2080,2100)).mean('year')

# calculate the difference:
DIFF_SSP245 = ACCESS_CM2_SAT_ssp245_21C-ACCESS_CM2_SAT_baseline

# Plot it:
DIFF_SSP245.plot(cmap='RdBu_r',vmin=-10.,vmax=10.)
plt.title('ACCESS-CM2 2-4.5 SAT in 2080-2100 minus SAT in 1900-1950')


<font color='red'> 
    
22. Copy the code above to also plot the spatial structure of the surface temperature change averaged over the last 20 years of the 21st century (2080-2100) of the SSP5-8.5 scenario. Include both SSP2-4.5 and SSP5-8.5 temperature change plots in your final report. Describe the structures you see. Does the spatial structure look similar to the changes over the historical period that we looked at in Week 3? What about over the high-latitude Arctic and Antarctic regions? **[3 marks]** 
</font>

### Precipitation

Precipitation is a particularly important variable to consider due to its direct impact on human society. Let's start out by looking at the mean field from the historical ACCESS-CM2 simulation:

In [29]:
ACCESS_CM2_hist.PRCP.mean('year').plot(vmin=0.,vmax=10.)

As discussed in the lectures, the precipitation is high in regions where air rises as part of the atmospheric circulation. These regions are in the tropics (over warm water), particularly in the western Pacific where the warmest sea surface temperatures are. Air also rises in the latitudes between 40-60N. In contrast, in areas where air descends there is much less precipitation. This is the case in the mid-latitude desert regions, including over much of Australia.

How does the precipitation change in the SSP2-4.5 and SSP5-8.5 scenarios according to the ACCESS-CM2 model?

<font color='red'> 
    
23. To answer this question, copy the code you made above for the surface temperature changes in 2080-2100 compared to that in 1900-1950 and change it in order to plot the change in precipitation instead (for both SSP2-4.5 and SSP5-8.5 scenarios). Include this plot in your final report **[2 marks]** 

Using this plot, answer the following questions:

24. Describe the changes you see. Which regions show increased precipitation and which regions show decreased precipitation? How does the magnitude of the change compare to the historical precipitation? **[2 marks]**
    
25. What is the global mean change in precipitation? To answer this plot a time series of the global average precipitation in the historical, SSP2-4.5 and SSP5-8.5 simulations using the `global_average` function. If you need to you can copy some of the code fro the GMST time series plot above (*However; you do not need to subtract any baseline precipitation from the historical simulation in this case*). Include this plot in your report. Explain physically where this change may be coming from (if you're stuck, do some of your own research on changes in the atmospheric water cycle)? **[3 marks]**
    
26. The changes we are looking at here are in annual mean precipitation. What kind of changes might you expect in rainfall extremes? Would you expect climate models such as ACCESS-CM2 to properly capture these changes? Why/why not?  **[2 marks]**
    
</font>

## Climate sensitivity: How do different models warm?

Since different climate models can respond differently to changes in greenhouse gases, lets take a look at how the ACCESS-CM2 results compare to two other climate models. We have provided data for two other models:

1. The Max-Planck Institut for Meteorology Earth System Model (MPI-ESM, here we use the "MPI-ESM1-2-LR" version).
2. The Canadian Earth System Model Version 5.0 (CanESM5).

The data for these models is contained within the `/course/data/` folder in the files:
- `MPI-ESM_hist_data.nc`, `MPI-ESM_ssp245_data.nc` and `MPI-ESM_ssp585_data.nc`
- `CanESM5_hist_data.nc`, `CanESM5_ssp245_data.nc` and `CanESM5_ssp585_data.nc`

<font color='red'>
    
27. By filling in the missing sections of code below (marked by a "..."), load and plot the Global Mean Surface Temperature change from the MPI-ESM and CanESM5 models over the historical period and for the SSP2-4.5 and SSP5-8.5 scenarios. Plot these curves along with the ACCESS-CM2 historical, SSP2-4.5 and SSP5-8.5 curves that you already plotted above. Include this plot in your final report. If you wish, you can plot these curves on several different panels/plots to make them easier to see.  **[3 marks]**
</font>

First load the data:

In [0]:
# Load historical, ssp245 and ssp585 datasets for MPI-ESM:
MPI_ESM_hist = xr.open_dataset('/course/data/MPI-ESM_hist_data.nc')
... # Add code for MPI-ESM SSP2-4.5 and SSP5-8.5 here.

# Load historical, ssp245 and ssp585 datasets for CanESM5:
... # Add code for CanESM5 historical, SSP2-4.5 and SSP5-8.5 here.


Then calculate the baseline GMST and then calculate the GMST change for each model (ACCESS-CM2 should already be completed above):

In [36]:
# Calculate the average temperature over the baseline period from the MPI-ESM historical simulation:
MPI_ESM_GMST_baseline = calc_baseline(global_average(MPI_ESM_hist.SAT,MPI_ESM_hist.area))

# Calculate GMST change in MPI-ESM in historical, SSP2-4.5 and SSP5-8.5 simulations:
MPI_ESM_hist_GMST = global_average(MPI_ESM_hist.SAT,MPI_ESM_hist.area)-MPI_ESM_GMST_baseline
... # Add code for MPI-ESM SSP2-4.5 and SSP5-8.5 here.

In [0]:
# Calculate the average temperature over the baseline period from the CanESM5 historical simulation:
... # Add code for CanESM5 here

# Calculate GMST change in CanESM5 in historical, SSP2-4.5 and SSP5-8.5 simulations:
... # Add code for CanESM5 here

Finally, make a plot:

In [37]:
# Plot the curves on one plot:
fig = plt.figure(figsize=(15,8))

# ACCESS-CM2 simulations:
ACCESS_CM2_hist_GMST.plot(label='ACCESS-CM2 historical',color='r')
ACCESS_CM2_ssp245_GMST.plot(label='ACCESS-CM2 SSP2-4.5',color='r',linestyle='--')
ACCESS_CM2_ssp585_GMST.plot(label='ACCESS-CM2 SSP5-8.5',color='r',linestyle=':')

# MPI-ESM simulations:
... # Add code here to plot the MPI-ESM GMST time series using blue lines, "color='b'"

# CanESM5 simulations:
... # add code here to plot the CanESM5 GMST time series using green lines "color='g'"

plt.title('Global Mean Surface Temperature')
plt.legend()
plt.grid() # Add grid lines

    
Using your plot/s, answer the following questions:

<font color='red'>

28. Which model warms the most and which the least? What is the magnitude of the difference? **[2 marks]**
29. Have a (brief - no need to read the whole thing) read of the carbonbrief.org article "CMIP6: the next generation of climate models explained" (https://www.carbonbrief.org/cmip6-the-next-generation-of-climate-models-explained/). In particular, the "Many higher sensitivity models" section. This section has a figure quantifying the climate sensitivity of all the CMIP6 models. Where do the CanESM5, MPI-ESM and ACCESS-CM2 climate models sit in this group? With that knowledge, discuss whether the range of possible GMST responses shown in your figure is a good representation of our uncertainty in that warming rate (according to the CMIP6 models as a whole). **[2 marks]**
30. Why might the models have different climate sensitivities, and thus warming rates? **[3 marks]**
</font>

## Regional change: Sydney and a location of your choice

As a final exercise, let's have a look at some time series of changes at specific locations, starting with Sydney.

<font color='red'>
    
31. The below code cell (copied from the "Bonus Material" section at the end of the Week 3 notebook) plots a time series of the historical SAT in Sydney. Add a few lines to also plot the SAT from the SSP2-4.5 and SSP5-8.5 scenarios. Then, copy the code from this block to a new code block and modify it to also plot the precipitation in Sydney from the three scenarios. Include these plots in your final report. Describe the changes you see. In particular, describe the magnitude of the change relative to the interannual variability in each variable (SAT and precipitation). Which variable, SAT or precipitation, shows the most robust change? Where could this interannual variability be coming from? **[3 marks]**

32. Choose another location around the Planet that you're interested in. Find its coordinates and then plot the changes in surface temperature and precipitation at the location. Include this figure in your final report and describe what it shows and the reasons behind these changes using what you have learnt above. **[2 marks]**
</font>

In [1]:
# Sydney longitude and latitude:
lon = 151.2
lat = -33.8

# Load SAT at this location using xarray's .sel command:
ACCESS_CM2_SAT_Sydney = ACCESS_CM2_hist.SAT.sel(longitude=lon,latitude=lat,method='nearest')

# Plot:
ACCESS_CM2_SAT_Sydney.plot(label='ACCESS-CM2')
plt.title('SAT changes near Sydney')
plt.legend()

## Bonus material

That concludes the main/assessed tasks for the Weeks2-4 practicals. I hope you enjoyed the exercises!

If you're interested please feel free to further explore the data. The following are a few things you could look at:
- Global and regional sea level changes in the different models and SSP scenarios.
- Changes in the radiative (shortwave and longwave) parameters in the SSP scenarios.
- Albedo changes.